# Medical Text

Medical Text Dataset [https://www.kaggle.com/datasets/chaitanyakck/medical-text/data]

In [1]:
import pandas as pd
import nltk
from spacy import displacy
import spacy
from transformers import pipeline
import pickle
import json
import os

C:\Users\claud\PycharmProjects\MedicalTextNLP\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\claud\PycharmProjects\MedicalTextNLP\.venv\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
C:\Users\claud\PycharmProjects\MedicalTextNLP\.venv\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
df = pd.read_csv('data/train.dat', sep="\t", header=None)

In [3]:
df.rename(columns={0:'condition', 1:'abstract'}, inplace=True)
df.head()

,condition,abstract
0,4,Catheterization laboratory events and hospital...
1,5,Renal abscess in children. Three cases of rena...
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...
3,5,Subclavian artery to innominate vein fistula a...
4,4,Effect of local inhibition of gamma-aminobutyr...


## Lexical Analysis
Lexical analysis consists of the operations of:
- **Sentence Splitting**: The technique aims to identify the beginning and end of a textual fragment (sentence or clause) with informative content, even if simple. To achieve this, it uses orthographic features of words (e.g., uppercase initial letters) and delimiters (e.g., punctuation).
- **Tokenization**: The goal of tokenization is to pinpoint the starting and ending positions of each token, whether it’s a word, a number, or a combination of symbols. As with sentence splitting, the process relies on orthographic features (e.g., initial capital letters) and delimiters (e.g., punctuation).
- **Lemmatization**: Post-tokenization techniques address the morphological analysis of word-tokens. Lemmatization identifies the base form (lemma) of inflected words, preserving their meaning and grammatical category. For example, the token _liked_ maps to the lemma *like*. This process minimizes lexical variation by consolidating different forms of the same word into a unified representation.
- **Stemming**: Like lemmatization, processes inflected forms but reduces them to their root, which may not correspond to a dictionary word. Unlike lemmatization, it focuses on inflections that create new words and may change the grammatical class, such as *probable* (adjective) stemming to *probably* (adverb).
- **POS Tagging**: Part-of-speech (POS) tagging assigns a grammatical category to each token, such as noun, verb, or adjective

It focuses on the main components of a text (words), and aims to recognize them in relation to the context in which they are used, such as sentences or clauses.

#### Funzioni utili
From now on there will be repetitive steps in order to check files existence prior to create each one: the following functions aims to simplify the process.

In [4]:
def check_existence(obj):
    """
    checks if we already stored the file
    Arg:
    obj = str name of the object
    """
    path = 'objects/'+obj+'.pkl'
    if os.path.exists(path):
        with open(path, 'rb') as file:
            file = pickle.load(file)
            return file
    else:
        return False

In [5]:
def save_step(name, obj):
    """
    Saves object to pickle and json files
    Args:
    name = 'obj_name'
    obj = object to save
    """

    with open('objects/'+name+'.pkl', 'wb') as file:
        pickle.dump(obj, file)

    with open('objects/'+name+'.json', 'w') as file:
        json.dump(obj, file)

In [6]:
lexical_df = df.copy() # a df to compute lexical analysis on

### Sentence Splitting, Word Tokenization, Lemmatization, Stemming and POS Tagging


In [7]:
# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

In [8]:
if check_existence('sentences'): # if the first obj exists for sure we created the others(?)
    sentences = check_existence('sentences')
    words = check_existence('words')
    lemmatization = check_existence('lemmatization')
    stemming = check_existence('stemming')
    pos_tag = check_existence('pos_tag')

else:
    sentences = []
    words = []
    lemmatization = []

    stemming = []
    porterStemmer = nltk.PorterStemmer()

    pos_tag = []

    for record in df["abstract"]:
        doc = nlp(record)
        split_record = []
        tokenized_record = []
        lemmatized_record = []
        stemmed_record = []
        pos_tag_record = []

        for sentence in doc.sents:
            split_record.append([sentence.text]) # Sentence Splitting
            tokenized_sent = []
            lemmatized_sent = []
            stemmed_sent = []
            pos_tag_sent = []

            for token in sentence:
                word = token.text # Tokenization
                tokenized_sent.append(word)

                lemmatized_sent.append(token.lemma_) # Lemmatization

                stemmed_sent.append(porterStemmer.stem(word)) # Stemming

                pos_tag_sent.append((word,token.tag_)) # POS Tagging

                # print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            tokenized_record.append(tokenized_sent)
            lemmatized_record.append(lemmatized_sent)
            stemmed_record.append(stemmed_sent)
            pos_tag_record.append(pos_tag_sent)

        sentences.append(split_record)
        words.append(tokenized_record)
        lemmatization.append(lemmatized_record)
        stemming.append(stemmed_record)
        pos_tag.append(pos_tag_record)

    # Save each step to a file
    save_step('sentences',sentences)
    save_step('words',words)
    save_step('lemmatization',lemmatization)
    save_step('stemming',stemming)
    save_step('pos_tag',pos_tag)

In [9]:
lexical_df["sentences"] = sentences
lexical_df['words'] = words
lexical_df['lemmatization'] = lemmatization
lexical_df['stemming'] = stemming
lexical_df['pos_tag'] = pos_tag

In [10]:
lexical_df.head()

,condition,abstract,sentences,words,lemmatization,stemming,pos_tag
0,4,Catheterization laboratory events and hospital...,[[Catheterization laboratory events and hospit...,"[[Catheterization, laboratory, events, and, ho...","[[catheterization, laboratory, event, and, hos...","[[catheter, laboratori, event, and, hospit, ou...","[[(Catheterization, NN), (laboratory, NN), (ev..."
1,5,Renal abscess in children. Three cases of rena...,"[[Renal abscess in children.], [Three cases of...","[[Renal, abscess, in, children, .], [Three, ca...","[[renal, abscess, in, child, .], [three, case,...","[[renal, abscess, in, children, .], [three, ca...","[[(Renal, JJ), (abscess, NN), (in, IN), (child..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,[[Hyperplastic polyps seen at sigmoidoscopy ar...,"[[Hyperplastic, polyps, seen, at, sigmoidoscop...","[[hyperplastic, polyp, see, at, sigmoidoscopy,...","[[hyperplast, polyp, seen, at, sigmoidoscopi, ...","[[(Hyperplastic, JJ), (polyps, NNS), (seen, VB..."
3,5,Subclavian artery to innominate vein fistula a...,[[Subclavian artery to innominate vein fistula...,"[[Subclavian, artery, to, innominate, vein, fi...","[[subclavian, artery, to, innominate, vein, fi...","[[subclavian, arteri, to, innomin, vein, fistu...","[[(Subclavian, JJ), (artery, NN), (to, IN), (i..."
4,4,Effect of local inhibition of gamma-aminobutyr...,[[Effect of local inhibition of gamma-aminobut...,"[[Effect, of, local, inhibition, of, gamma, -,...","[[effect, of, local, inhibition, of, gamma, -,...","[[effect, of, local, inhibit, of, gamma, -, am...","[[(Effect, NN), (of, IN), (local, JJ), (inhibi..."


In [11]:
sentences[0][0]

['Catheterization laboratory events and hospital outcome with direct angioplasty for acute myocardial infarction To assess the safety of direct infarct angioplasty without antecedent thrombolytic therapy, catheterization laboratory and hospital events were assessed in consecutively treated patients with infarctions involving the left anterior descending (n = 100 patients), right (n = 100), and circumflex (n = 50) coronary arteries.']

### Stop-words Removal
Stop-words are common words that do not carry specific meaning, such as articles, prepositions, and conjunctions.
It is usually performed after lexical analysis to avoid inaccuracies in subsequent syntactic or semantic analyses.

In [12]:
# nltk.download('stopwords')

In [13]:
if check_existence('stopwords_removal'):
    stopwords_removal = check_existence('stopwords_removal')

else:
    stopwords = nltk.corpus.stopwords.words('english')

    stopwords_removal = []

    for record in lexical_df['sentences']:
        filtered_record = []
        for sentence in record:
            filtered_sentence = [word for word in sentence if word.lower() not in stopwords]
            filtered_record.append(filtered_sentence)

        stopwords_removal.append(filtered_record)

    save_step('stopwords_removal',stopwords_removal)

In [14]:
lexical_df["stopwords_removal"] = stopwords_removal

In [15]:
lexical_df.head()

,condition,abstract,sentences,words,lemmatization,stemming,pos_tag,stopwords_removal
0,4,Catheterization laboratory events and hospital...,[[Catheterization laboratory events and hospit...,"[[Catheterization, laboratory, events, and, ho...","[[catheterization, laboratory, event, and, hos...","[[catheter, laboratori, event, and, hospit, ou...","[[(Catheterization, NN), (laboratory, NN), (ev...",[[Catheterization laboratory events and hospit...
1,5,Renal abscess in children. Three cases of rena...,"[[Renal abscess in children.], [Three cases of...","[[Renal, abscess, in, children, .], [Three, ca...","[[renal, abscess, in, child, .], [three, case,...","[[renal, abscess, in, children, .], [three, ca...","[[(Renal, JJ), (abscess, NN), (in, IN), (child...","[[Renal abscess in children.], [Three cases of..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,[[Hyperplastic polyps seen at sigmoidoscopy ar...,"[[Hyperplastic, polyps, seen, at, sigmoidoscop...","[[hyperplastic, polyp, see, at, sigmoidoscopy,...","[[hyperplast, polyp, seen, at, sigmoidoscopi, ...","[[(Hyperplastic, JJ), (polyps, NNS), (seen, VB...",[[Hyperplastic polyps seen at sigmoidoscopy ar...
3,5,Subclavian artery to innominate vein fistula a...,[[Subclavian artery to innominate vein fistula...,"[[Subclavian, artery, to, innominate, vein, fi...","[[subclavian, artery, to, innominate, vein, fi...","[[subclavian, arteri, to, innomin, vein, fistu...","[[(Subclavian, JJ), (artery, NN), (to, IN), (i...",[[Subclavian artery to innominate vein fistula...
4,4,Effect of local inhibition of gamma-aminobutyr...,[[Effect of local inhibition of gamma-aminobut...,"[[Effect, of, local, inhibition, of, gamma, -,...","[[effect, of, local, inhibition, of, gamma, -,...","[[effect, of, local, inhibit, of, gamma, -, am...","[[(Effect, NN), (of, IN), (local, JJ), (inhibi...",[[Effect of local inhibition of gamma-aminobut...


# Syntax Analysis
Syntax analysis consists of:
- Shallow Parsing
- Deep Parsing

### Shallow Parsing
Syntactic parsing extends chunking by generating a parse tree. This tree organizes POS-tagging results as leaf nodes and syntactic structures (often chunks) as intermediate nodes, connected hierarchically without representing specific relationships.


In [16]:
syntax_df = df.copy() # a df to compute syntax analysis on

In [17]:
if check_existence('chunking'):
    chunking = check_existence('chunking')

else:
    # Define the grammar and the chunk parser
    grammar = "NP: {<NNP><NNP>}"
    cp = nltk.RegexpParser(grammar) # chunk parser

    # Apply chunking to each record
    chunking = []
    for record in lexical_df['pos_tag']:
        chunked_record = [cp.parse(sentence) for sentence in record]

        chunking.append(chunked_record)

    save_step('chunking',chunking)

In [18]:
# Save the chunking results into the dataframe
syntax_df["shallow_parsing"] = chunking

In [19]:
# Display the dataframe
syntax_df.head()

,condition,abstract,shallow_parsing
0,4,Catheterization laboratory events and hospital...,"[[(Catheterization, NN), (laboratory, NN), (ev..."
1,5,Renal abscess in children. Three cases of rena...,"[[(Renal, JJ), (abscess, NN), (in, IN), (child..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,"[[(Hyperplastic, JJ), (polyps, NNS), (seen, VB..."
3,5,Subclavian artery to innominate vein fistula a...,"[[(Subclavian, JJ), (artery, NN), (to, IN), (i..."
4,4,Effect of local inhibition of gamma-aminobutyr...,"[[(Effect, NN), (of, IN), (local, JJ), (inhibi..."


In [20]:
# Display the chunking result for the first record
print(syntax_df['shallow_parsing'][0][4])
#syntax_df['chunking'][0][4]

(S
  There/EX
  was/VBD
  one/CD
  in/IN
  -/HYPH
  laboratory/NN
  death/NN
  (/-LRB-
  shock/NN
  patient/NN
  with/IN
  infarction/NN
  of/IN
  the/DT
  left/JJ
  anterior/JJ
  descending/VBG
  coronary/JJ
  artery/NN
  )/-RRB-
  ./.)


In [21]:
# To draw the parse tree
#syntax_df['chunking'][1][1].draw()

### Deep Parsing
Differently from _Shallow parsing_, _Deep parsing_ aims to infer dependency relationships between nodes.
The result is a dependency graph which relates words that are syntactically linked.

In [22]:
if check_existence('deep_parsing'):
    deep_parsing = check_existence('deep_parsing')

else:
    deep_parsing = []
    for sentences in syntax_df["abstract"]:
        sentence_dep = []
        doc = nlp(sentences)
        for token in doc:
            sentence_dep.append((str(token.text), str(token.dep_), str(token.head.text), str([child for child in token.children])))
            # creates a tuple containing the token, dependency nature, head and all dependents of the token
        deep_parsing.append(sentence_dep)

    save_step('deep_parsing',deep_parsing)

In [23]:
syntax_df["deep_parsing"] = deep_parsing

In [24]:
syntax_df.head()

,condition,abstract,shallow_parsing,deep_parsing
0,4,Catheterization laboratory events and hospital...,"[[(Catheterization, NN), (laboratory, NN), (ev...","[(Catheterization, compound, events, []), (lab..."
1,5,Renal abscess in children. Three cases of rena...,"[[(Renal, JJ), (abscess, NN), (in, IN), (child...","[(Renal, amod, abscess, []), (abscess, ROOT, a..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,"[[(Hyperplastic, JJ), (polyps, NNS), (seen, VB...","[(Hyperplastic, amod, polyps, []), (polyps, ns..."
3,5,Subclavian artery to innominate vein fistula a...,"[[(Subclavian, JJ), (artery, NN), (to, IN), (i...","[(Subclavian, amod, artery, []), (artery, ROOT..."
4,4,Effect of local inhibition of gamma-aminobutyr...,"[[(Effect, NN), (of, IN), (local, JJ), (inhibi...","[(Effect, ROOT, Effect, [of, :, study, .]), (o..."


In [25]:
# Seleziona la prima riga del dataframe
sentence = syntax_df["abstract"].iloc[0]

# Analizza la frase
doc = nlp(sentence)

# Visualizza il grafico di dipendenze
displacy.render(doc, style="dep", jupyter=True)

## Semantic Analysis

Semantic analysis aims to extract the meaning of a text, focusing on the relationships between entities and the context in which they appear.

### Entity Extraction

Entity extraction identifies named entities in a text, such as people, organizations, or locations.

In [26]:
semantic_df = df.copy() # a df to compute semantic analysis on

This code extracts named entities from the "abstract" column of a DataFrame called syntax_df using a NLP model.
For each record, it collects the entities and their labels, storing them in a list of tuples.

In [27]:
pipe = pipeline("token-classification", model="Clinical-AI-Apollo/Medical-NER", aggregation_strategy='simple')

C:\Users\claud\PycharmProjects\MedicalTextNLP\.venv\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\claud\PycharmProjects\MedicalTextNLP\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\claud\.cache\huggingface\hub\models--Clinical-AI-Apollo--Medical-NER. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate

In [28]:
# if the entities are already extracted, load them from the pickle file
if check_existence('entities'):
    entities = check_existence('entities')

else:
    entities = []
    for record in semantic_df["abstract"]:
        result = pipe(record)

        record_entities = [(entity['word'], entity['entity_group']) for entity in result]

        entities.append(record_entities)

    save_step('entities',entities)

In [29]:
semantic_df["entities"] = entities

To get a sense of the entities extracted from the text, we can display the unique entities found in the dataset.

In [30]:
# Flatten the list of entities and extract only the labels
all_labels = pd.Series([label for record in semantic_df["entities"] for _, label in record])

# Perform value counts on the labels
all_labels.value_counts()

DETAILED_DESCRIPTION      100224
DIAGNOSTIC_PROCEDURE       74383
SIGN_SYMPTOM               68825
LAB_VALUE                  68092
DISEASE_DISORDER           61689
BIOLOGICAL_STRUCTURE       55139
THERAPEUTIC_PROCEDURE      27167
MEDICATION                 19053
DATE                       15674
DURATION                    5689
DOSAGE                      5519
AGE                         3286
CLINICAL_EVENT              2792
DISTANCE                    2380
SEVERITY                    2013
NONBIOLOGICAL_LOCATION      1819
HISTORY                     1763
SEX                         1451
FAMILY_HISTORY               426
AREA                         139
ADMINISTRATION               129
COREFERENCE                  122
OTHER_ENTITY                  15
PERSONAL_BACKGROUND           13
VOLUME                         9
TIME                           4
Name: count, dtype: int64

In [31]:
semantic_df.head()

,condition,abstract,entities
0,4,Catheterization laboratory events and hospital...,"[(Catheterization laboratory events, DIAGNOSTI..."
1,5,Renal abscess in children. Three cases of rena...,"[(Renal, BIOLOGICAL_STRUCTURE), (abscess, DISE..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,"[(Hyperplastic polyps, SIGN_SYMPTOM), (sigmoid..."
3,5,Subclavian artery to innominate vein fistula a...,"[(Subclavian artery to innominate vein, BIOLOG..."
4,4,Effect of local inhibition of gamma-aminobutyr...,"[(dorsomedial hypothalamus, BIOLOGICAL_STRUCTU..."


In [32]:
def display_entities(pipe, document):
    """
    Mostra le entità riconosciute utilizzando displacy.render().

    Parametri:
        pipe: Pipeline Hugging Face per il riconoscimento delle entità.
        document (str): Testo da processare.

    Ritorna:
        - Visualizzazione delle entità tramite displacy.
        - Lista di entità e relative etichette.
    """
    # Esegui la pipeline sul documento
    result = pipe(document)

    # Crea un modello SpaCy vuoto per gestire il documento
    nlp = spacy.blank("en")
    doc = nlp.make_doc(document)

    # Aggiungi le entità manualmente
    ents = []
    for entity in result:
        start_char = entity['start']
        end_char = entity['end']
        label = entity['entity_group']
        # Crea manualmente uno span
        span = doc.char_span(start_char, end_char, label=label, alignment_mode="expand")
        if span is not None:
            ents.append(span)

    # Assegna le entità al documento
    doc.ents = ents

    # Visualizza con displacy
    displacy.render(doc, style='ent', jupyter=True)

In [33]:
display_entities(pipe, semantic_df["abstract"].iloc[1])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [34]:
semantic_df.head()

,condition,abstract,entities
0,4,Catheterization laboratory events and hospital...,"[(Catheterization laboratory events, DIAGNOSTI..."
1,5,Renal abscess in children. Three cases of rena...,"[(Renal, BIOLOGICAL_STRUCTURE), (abscess, DISE..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,"[(Hyperplastic polyps, SIGN_SYMPTOM), (sigmoid..."
3,5,Subclavian artery to innominate vein fistula a...,"[(Subclavian artery to innominate vein, BIOLOG..."
4,4,Effect of local inhibition of gamma-aminobutyr...,"[(dorsomedial hypothalamus, BIOLOGICAL_STRUCTU..."


### Relation Extraction

Relation extraction identifies the connections between entities in a text, such as the subject, object, and verb of a sentence.

This process is more complex than entity extraction, as it requires understanding the syntactic structure of the text to infer relationships between entities.

1. **Iterate through sentences**:
   Process each sentence (`doc`) and its entities, storing entities in a set for faster lookup (`entities_set`).

2. **Extract relations from tokens**:
   Loop through tokens with dependencies like "ROOT" or "VERB" to identify:
   - **Subjects**: Found in children with dependencies like "nsubj" or "agent."
   - **Objects**: Found in children with dependencies like "dobj" or "pobj."

3. **Record relations**:
   - Direct relations: `(subject, verb, object).`
   - Prepositional relations: Handle `prep` and `pobj` to form `(subj, verb_prep, obj)` or similar.

4. **Handle conjunctions**:
   Add relations involving conjunctive tokens (`conj`).



In [35]:
if check_existence('relations'):
    all_relations = check_existence('relations')

else:
    # Relations Extraction
    all_relations = []

    for sentences, sentence_entities in zip(semantic_df["abstract"], semantic_df["entities"]):
        doc = nlp(sentences)
        sentence_relations = []
        entities_set = {ent_text for ent_text, _ in sentence_entities}  # Use a set for faster lookup

        # Extract relations for each root or verb token
        for token in doc:
            if token.dep_ in ("ROOT", "VERB", "aux", "auxpass"):
                # Find the subject
                subj = None
                for child in token.children:
                    if child.dep_ in ("nsubj", "nsubjpass", "csubj", "agent"):
                        for ent_text in entities_set:
                            if ent_text in child.text or child.text in ent_text:
                                subj = ent_text
                                break

                # Find the object
                obj = None
                for child in token.children:
                    if child.dep_ in ("dobj", "pobj", "iobj", "attr", "acomp"):
                        for ent_text in entities_set:
                            if ent_text in child.text or child.text in ent_text:
                                obj = ent_text
                                break

                # Record direct relations
                if subj and obj:
                    sentence_relations.append((subj, token.lemma_, obj))

                # Handle prepositional relations
                for child in token.children:
                    if child.dep_ == "prep":
                        for pobj in child.children:
                            if pobj.dep_ == "pobj":
                                for ent_text in entities_set:
                                    if ent_text in pobj.text or pobj.text in ent_text:
                                        if subj:
                                            sentence_relations.append((subj, f"{token.lemma_}_{child.text}", ent_text))
                                        elif obj:  # Use the object if no subject
                                            sentence_relations.append((obj, f"{token.lemma_}_{child.text}", ent_text))

                # Handle conjunctive subjects and objects
                for child in token.children:
                    if child.dep_ == "conj" and subj:
                        sentence_relations.append((subj, token.lemma_, child.text))

        # Post-process to remove duplicates
        sentence_relations = list(set(sentence_relations))

        # Add relations to the main list
        all_relations.append(sentence_relations)

    save_step('relations',all_relations)

In [36]:
all_relations

[[('Cardiogenic shock', 'be_in', 's'),
  ('s', 'be', 's'),
  ('s', 'assess_in', 's'),
  ('s', 'occur_in', 's'),
  ('s', 'occur_with', 'myocardial infarction'),
  ('s', 'assess_with', 's'),
  ('s', 'assess_with', 'infarct'),
  ('s', 'assess', 'arteries'),
  ('s', 'assess_with', 'infarction'),
  ('s', 'occur_with', 'infarct'),
  ('s', 'occur_with', 'infarction'),
  ('Cardiogenic shock', 'be', 's')],
 [('Fever', 'be', 'associated'),
  ('Antibiotics', 'produce', 'required'),
  ('Reflux', 'document', 'had')],
 [('Hyperplastic polyps', 'remove_at', 'sigmoidoscopy'),
  ('Hyperplastic polyps', 'remove_at', 'sigmoid'),
  ('Hyperplastic polyps', 'remove_at', 'oid'),
  ('adenoma', 'have_at', 'proximal sites')],
 [('Subclavian artery to innominate vein', 'innominate', 'fistula'),
  ('Subclavian artery to innominate vein', 'innominate_after', 's')],
 [('microdialysis', 'cause', 'variable increases'),
  ('perfusion', 'cause', 'variable increases'),
  ('microdialysis', 'cause', 'change'),
  ('variabl

In [37]:
# Add the relations list as a column to semantic_df
semantic_df['relations'] = [relations if relations else pd.NA for relations in all_relations]

In [38]:
# Display the updated dataframe
semantic_df[semantic_df['relations'].notna()]

,condition,abstract,entities,relations
0,4,Catheterization laboratory events and hospital...,"[(Catheterization laboratory events, DIAGNOSTI...","[(Cardiogenic shock, be_in, s), (s, be, s), (s..."
1,5,Renal abscess in children. Three cases of rena...,"[(Renal, BIOLOGICAL_STRUCTURE), (abscess, DISE...","[(Fever, be, associated), (Antibiotics, produc..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,"[(Hyperplastic polyps, SIGN_SYMPTOM), (sigmoid...","[(Hyperplastic polyps, remove_at, sigmoidoscop..."
3,5,Subclavian artery to innominate vein fistula a...,"[(Subclavian artery to innominate vein, BIOLOG...","[(Subclavian artery to innominate vein, innomi..."
4,4,Effect of local inhibition of gamma-aminobutyr...,"[(dorsomedial hypothalamus, BIOLOGICAL_STRUCTU...","[(microdialysis, cause, variable increases), (..."
...,...,...,...,...
14430,5,Extensive peripheral retinectomy combined with...,"[(Extensive, DETAILED_DESCRIPTION), (periphera...","[(normotensive, be_Of, eyes), (retinal detachm..."
14432,4,Embolization with particles in thoracic intram...,"[(Embolization, THERAPEUTIC_PROCEDURE), (parti...","[(up, be, 1-15 years), (Clinical aggravation, ..."
14434,1,Mammographic measurements before and after aug...,"[(Mammographic measurements, DIAGNOSTIC_PROCED...","[(Anterior breast tissue, see_with, compressio..."
14436,2,Complications of Tenckhoff catheters post remo...,"[(ckhoff, DISEASE_DISORDER), (January 1979 to,...","[(D, include, monitored), (peritoneal dialysis..."
